# Final Project 
Jesus Olivera, Pujita Ravichandar

## Introduction
<br>
Climate change poses a great risk to humanity. Thermometer readings around the world have been rising, and the causes are attributed to a blend of human activity and some natural variability.
<br>


Climate change poses a greater risk to Small Island Developing States (SIDS) and Least Developed Countries (LDCs). They are least able to recover from climate stresses and their economic growth is highly dependent on climate-sensitive sectors.
<br>


Between 1970 and 2019, 79% of natural disasters worldwide involved weather and climate related hazards. It is expected that climate change will threaten years of development progress and increase the number of people living in poverty, adding as many as 122 million more people by 2030. 
<br>


Scientists have high confidence that the global temperatures will continue to rise for decades to come, largely due to the greenhouse gases produced by human activity.
<br>


In our research we seek to understand the relationship between temperature anomalies, glaciers mass and deaths by natural disasters.

## Research Questions
<br>
Our research topic can be summarized in the following sentence. How are global temperatures affecting the weather and safety of our planet? 

**This question can be broken down into the following sub questions:**

<br>

* What are the trends in global surface temperature?
* How do global temperature trends affect glaciers?
* How do global temperature trends affect the number of deaths by natural disasters?


### Importing all libraries to be used

In [2]:
import requests
import pandas as pd
import json
import urllib.request
from lxml import objectify
from io import StringIO
from pprint import pprint
from pandas import DataFrame
import numpy as np
import re
import csv
from bs4 import BeautifulSoup
from html.parser import HTMLParser
from urllib import request
import lxml.html as lh
import urllib
import matplotlib.pyplot as plt
plt.style.use('default')
import seaborn as sns
import scipy

import geopandas as gpd

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

ModuleNotFoundError: No module named 'geopandas'

## Phase 1 - Data Acquisition, Data Preparation & Exploratory Data Analysis


### Data Acquisition: Surface Temperature  Anomaly

There are five data sources used for this project. The data acquitison, preparation, and EDA are detailed below.
<br>


1. Our World Data: Surface temperature data. GISTEMP surface temperature anomaly, 2017. The temperature data is measured in degrees Celsius and it is relative to the 1951 – 1980 global average temperature. The data is based on the GISTEMP analysis from the NASA Goddard Institute for Space Studies. From this source we extracted necessary data regarding global surface temperature by region and year. The data was collected from this source via csv.

**GISTEMP surface temperature anomaly, 2017** (https://ourworldindata.org/grapher/surface-temperature-anomaly-gistemp?time=2017).
<br>
 
 
Explanation of "Surface Temperature Anomaly": The land surface temperature can be compared to how warm or cool the earth feels if you were to touch it. From the satellite's point of view, it is what it sees when it looks at the ground through the atmosphere. There is a difference between land an air surface temperature. Here, an anomaly can be defined as an instance when conditions depart from the average conditions of a particular place at a given time of the year. For our dataset, the anomaly values are given with a country and a year.  Surface temperature anomalies are very insightful for learning about weather patterns, environmental changes, and global warming, among others.

In [ ]:
# GISTEMP surface temperature anomaly, 2017

read = pd.read_csv("https://raw.githubusercontent.com/jaynuel/AIM-5001/master/surface-temperature-anomaly-gistemp.csv")
#read.head(3)


### Data Preparation: Global Surface Temperature Anomaly


In [ ]:
# Looking for null values
print(read.isnull().sum())

# Shape of the data
print(read.shape)

In [ ]:
# Finding null values 
is_nan = read.isnull()
row_with_nan = is_nan.any(axis=1)
nan_rows = read[row_with_nan]

# Finding unique countries with "NaN" country code
nan_rows.Entity.unique()

The dataframe is missing the 3 digit country code for Micronesia. To replace the "NaN" values for "FSM" the function "replace" was applied.

In [ ]:
# Replacing "NaN" values for Micronesia 3 digit ISO code: FSM

read['Code'] = read['Code'].replace(np.NaN, 'FSM')

# Reviewing if "NaN" values were replaced 
read.isnull().sum()

In [ ]:
# Confirming updates were applied
micronesia_code = read.loc[read['Code']== 'FSM']
print(micronesia_code)

# Confirming all codes are correct
print(read.Code.unique())


As results show above, some country codes have "OWID_" before the country code. To update the dataframe the function "replace" was used.

In [ ]:
# Replacing "OWID_" for actual country code
read = read.replace('OWID_KOS','KOS')
read = read.replace('OWID_CYN','CYN')

# Confirming values were replaced
read.Code.unique()

### Exploratory Data Analysis: Surface Temperature  Anomaly


In [ ]:
# Analyzing the dataset using the describe function to get a statistical summary
read.describe().round(2)

In [ ]:
# Understanding the distribution of the dataset per country
read['Entity'].value_counts().sort_values()

In [ ]:
# Groupping by 'year'
temp_country = read.groupby(['Year']).mean()

sns.scatterplot(data = temp_country)
plt.title("Global Average Surface Temperature Anomaly", size=15)
plt.show()

### Exploratory Data Analysis: Global Surface Temperature Anomaly - Results

Based on the scatter plots above and the choropleth map coming up in the bext phase, this dataset contains information about global surface temerpature anomalies ranging from 1881 to 2017 for 198 countries. Each country has a surface temperature anomaly value for this range of years. The scatter plot below shows the average surface temperature anomalies of all the countries by year. This graph shows a sharp increase in the global surface temperature over time. There is a particularly steep increase from 1980 onwards. Especially compared to the years of 1880 to 1980, we can clearly see an increase in the global surface temperature trends.

https://earthobservatory.nasa.gov/global-maps/MOD_LSTAD_M

### Data Acquisition: Deaths Caused By Natural Disasters


2. Deaths caused by natural distasters by country and year. The data set contains the name of the country, the country code, the year and the total deaths due to natural disasters. The data needed was collected from this source via csv. (https://ourworldindata.org/natural-disasters)

**CSV:**https://raw.githubusercontent.com/jaynuel/AIM-5001/master/deaths-natural-disasters-ihme.csv


In [ ]:
# Deaths by natural disasters
read1 = pd.read_csv("https://raw.githubusercontent.com/jaynuel/AIM-5001/master/deaths-natural-disasters-ihme.csv")
deaths_by_country = read1.rename(columns={"Deaths - Exposure to forces of nature - Sex: Both - Age: All Ages (Number)":"Total_Deaths"})

# Printing Results
deaths_by_country.head(3)

### Data Preparation: Deaths Caused By Natural Disasters

In [ ]:
# Looking for null values
print(deaths_by_country.isnull().sum())

# Shape of the data
print(deaths_by_country.shape)

In [ ]:
# Finding null values 
nan_values = deaths_by_country[deaths_by_country['Code'].isna()]
nan_values.Entity.unique()

Results above show that the reason why the dataframe contains "NaN" values on the "Code" column is because the values were also classified per region. To make sure that the dataframe is consistent, all the rows containing "NaN" values will be dropped from the "death_by_country" dataframe.

In [ ]:
# Dropping rows containing "NaN" values from "deaths_by_country" dataframe and 
# re-seting index values
deaths_by_country = deaths_by_country.dropna().reset_index(drop=True)

# Confirming "NaN" values were dropped from "deaths_by_country"
print(deaths_by_country.isnull().sum())


In [ ]:
# Removing groupped values to ensure only individual country values are contained in dataframe 
# to be used in analysis

deaths_by_country.drop(deaths_by_country[deaths_by_country['Entity']== 'World'].index, inplace=True)


### Exploratory Data Analysis: Deaths Caused By Natural Disasters

In [ ]:
# Analyzing the dataset using the describe function to get a statistical summary

# Printing bold title
print('\033[1m'+'Deaths by Country'+'\033[0m')
# Results
print(deaths_by_country.describe().round(2))


In [ ]:
# Understanding the distribution of the dataset per country

# Printing bold title
print('\033[1m'+'Deaths by Country'+'\033[0m')
# Printing Results
print(deaths_by_country['Entity'].value_counts().sort_values().head(5))


In [ ]:
# Analysing the distribution of the dataset "deaths_by_country" by year 
deaths_by_country.groupby('Year').Entity.agg(['count']).head(5)

In [ ]:
# Calculating the total deaths by year using the "deaths_by_country" dataset
total_by_country = deaths_by_country.groupby('Year').Total_Deaths.agg(['sum']).round()
total_by_year = deaths_by_country.groupby('Entity').Total_Deaths.agg(['sum']).round()

print(total_by_country.head(5))
print(total_by_year.head(5))

In [ ]:
# Visualization of total deaths by year using the "deaths_by_country" dataframe
sns.scatterplot(data=total_by_country, s=90)
plt.ylabel('Count', fontsize=10)
plt.title('Total Death Count Grouped by Year Due to Natural Disasters', fontsize=11)
plt.show()

## Understanding the Outliers

In [ ]:
# Returning largest values per all years to analyse outliers
outliers_deaths = total_by_country.nlargest(4, 'sum')

# Printing results
outliers_deaths

In [ ]:
# Returning countries with the largest number of deaths by natural disasters
total_by_year.nlargest(4, 'sum')

In [ ]:
# Printing rows containing results + bold titles. A line break was printed for 
# an easier results reading

print('\033[1m' +'Haiti: 2010 Earthquake'+'\033[0m')
print(deaths_by_country.loc[deaths_by_country['Entity']=='Haiti'].nlargest(1,'Total_Deaths'))

print('')
print('\033[1m' +'Indonesia: 2004 Indian Ocean Earthquake & Tsunami'+'\033[0m')
print(deaths_by_country.loc[deaths_by_country['Entity']=='Indonesia'].nlargest(1,'Total_Deaths'))

print('')
print('\033[1m' +'Bangladesh: 1991 Cyclone'+'\033[0m')
print(deaths_by_country.loc[deaths_by_country['Entity']=='Bangladesh'].nlargest(1,'Total_Deaths'))

print('')
print('\033[1m' +'Myanmar: 2008 Cyclone Nargis'+'\033[0m')
print(deaths_by_country.loc[deaths_by_country['Entity']=='Myanmar'].nlargest(1,'Total_Deaths'))


In [ ]:
deaths_by_country.head()

### Exploratory Data Analysis: Deaths Caused by Natural Disasters - Results

Based on the scatter plot above, this dataset contains information about the deaths caused by natural disasters from 1990 to 2017. Each country has a number of total deaths by natural disasters for this range of years. The scatter plot below shows the sum od deaths for all of the countries by year. This graph does not have a steady increasing trend like the surface temperature anomaly graphs. However, there are significant peaks in this graphs that are interesting to look at. These peaks happen during the years of 1991, 2004, 2008, and 2010 and are the results of very damaging natural disasters. The details of these events are below.

The peaks of this dataset reflects the devastating effects of 4 natural disasters:

1. **Bangladesh** 1991 cyclone: The cyclone made landfall in southeastern Bangladesh around the time of high tide, which was already 5.5 m (18 ft) above normal; in addition, the cyclone produced a 6.1 m (20 ft) storm surge that inundated the coastline. The storm also brought winds of around 240 km/h (150 mph). Winds exceeding 220 km/h (137 mph) lashed a populated region of the coast for about 12 hours, as well as 12 offshore islands.(https://en.wikipedia.org/wiki/1991_Bangladesh_cyclone)
<br>

2. **Indonesia** 2004 Indian Ocean Earthquake & Tsunami:The 2004 Indian Ocean earthquake and tsunami (also known as the Boxing Day Tsunami and, by the scientific community, the Sumatra–Andaman earthquake) occurred at 07:58:53 in local time (UTC+7) on 26 December, with an epicentre off the west coast of northern Sumatra, Indonesia. It was an undersea megathrust earthquake that registered a magnitude of 9.1–9.3 Mw, reaching a Mercalli intensity up to IX in certain areas. The earthquake was caused by a rupture along the fault between the Burma Plate and the Indian Plate.(https://en.wikipedia.org/wiki/2004_Indian_Ocean_earthquake_and_tsunami)
<br>

3. **Myanmar** 2008 Cyclone Nargis: Extremely Severe Cyclonic Storm Nargis was an extremely destructive and deadly tropical cyclone that caused the worst natural disaster in the recorded history of Myanmar during early May 2008. The cyclone made landfall in Myanmar on Friday, 2 May 2008, sending a storm surge 40 kilometres up the densely populated Irrawaddy delta, causing catastrophic destruction. (https://en.wikipedia.org/wiki/Cyclone_Nargis)
<br>

4. **Haiti** 2010 Earthquake: The 2010 Haiti earthquake (French: Séisme de 2010 à Haïti; Haitian Creole: Tranblemanntè 12 janvye 2010 nan peyi Ayiti) was a catastrophic magnitude 7.0 Mw earthquake, with an epicenter near the town of Léogâne (Ouest) and approximately 25 kilometres (16 mi) west of Port-au-Prince, Haiti's capital. The earthquake occurred at 16:53 local time (21:53 UTC) on Tuesday, 12 January 2010.(https://en.wikipedia.org/wiki/2010_Haiti_earthquake) 




### Data Acquisition: Glacier Mass Balance

3. DataHub.io: Core/Glacier-Mass-Balance. Average cumulative balance of “reference” glaciers worldwide from 1945-2014 from US EPA and World Glacier Monitoring Service (WGMS). This is cumulative change in mass balance of set of “reference” glaciers worldwide. The values represent the average of all the glaciers that were measured. Negative values indicate a net loss of ice and snow compared with the base year of 1945.From this source we will be able to extract necessary data regarding glaciers mass worldwide by year. The data needed will be collected from this source via Web Scraping.

**Datahub.io: Average cumulative mass balance of reference glaciers worldwide** (https://datahub.io/core/glacier-mass-balance#data).

In [ ]:
# Web Scapping Data
r = requests.get('https://pkgstore.datahub.io/core/glacier-mass-balance/glaciers_csv/data/c04ec0dab848ef8f9b179a2cca11b616/glaciers_csv.csv').text
data = StringIO(r)
glacier = pd.read_csv(data, sep=",")

# Printing dataframe
glacier.head(3)

### Data Preparation: Glacier Mass Balance


In [ ]:
# Looking for null values
print(glacier.isnull().sum())

# Shape of the data
print(glacier.shape)

In [ ]:
# Replacing "NaN" value for 0
glacier = glacier.replace(np.NaN,0)

# Confirming value was replaced
print(glacier.isnull().sum())
print(glacier.head(1))

### Exploratory Data Analysis: Glacier Mass Balance


In [ ]:
# Analyzing the dataset using the describe function to get a statistical summary
glacier.describe().round(2)

In [ ]:
# Visualizing changes of glaciers mean comulative mass balance by decade using Matplotlib

x = glacier[['Year']]
y = glacier[['Mean cumulative mass balance']]

g1 = plt.scatter(x,y)
g1 = plt.title('Changes in Glaciers Comulative Mass Balance By Decade')
g1 = plt.xlabel('Decade')
g1 = plt.ylabel('Change in Mean Cumulative Mass Balance')
g1 = plt.show()
g1


In [ ]:
glacier['Year'].min()

### Exploratory Data Analysis: Glacier Mass Balance - Results

Based on the scatter plot above, this dataset contains information about the mean cumulative mass balance of glaciers ranging from 1945 to 2014 for 1glaciers worldwide. Each year has information about the cumulative mass balance of the glaciers and the number of observations associated with this observation. The scatter plot above shows the the change in the mean cumulative mass balance by year. This graph shows a steady decrease in the mass of the glaciers over time. This intuatively fits with our EDA showing the steady increase of global surface temperature. There is a particularly steep decline from around 1980 onwards.

### Data Acquisition: Climate Projections


4. Climate Impact Lab: Daily climate projections based on representative concentration pathway 2.6, 4.5, and 8.5 (Van Vuuren et al., 2012) experiments run by global climate models participating in the Coupled Model Intercomparison Project Phase 5 (CMIPS) exercise (Taylor et al., 2012). The projections were aggregated to regional estimates by first transforming the daily min, average, or maximum temperatures at the grid scale, then aggregating to regions using a weighted average. From this source we will be able to extract necessary data regarding climate projections by region and year. The data needed was collected from this source as a CSV format.

**Climate Impact Lab: Global Map** (http://www.impactlab.org/map/#usmeas=absolute&usyear=1981-2010&gmeas=absolute&gyear=1986-2005&tab=global).

In [ ]:
# Climate projections
r3= requests.get('https://raw.githubusercontent.com/jaynuel/AIM-5001/master/Climate-Impact-Lab_Global-Data%20(2).csv')
r4 = r3.text
data = StringIO(r4)
projections = pd.read_csv(data, sep=",")

# Printing dataframe
projections.head(3)

### Data Preparation: Climate Projections

In [ ]:
# Looking for null values
print(projections.isnull().sum())

# Shape of the data
print(projections.shape)

In [ ]:
# Finding null values 
projections[projections['Country'].isna()]

As results above show, all rows containing null values can be drop. To accomplish this, the function "dropna" was used.

In [ ]:
# Dropping rows containing null values
projections = projections.dropna()

# Confirming all rows with null values were dropped
print(projections.isnull().sum())

To better analyze the projections, the original "projections" dataframe was divided in projections by a range of years. The column names were updated for a more meaningful interpretation.

In [ ]:
# Cleaned dataframes by projection year range
AVG_1986_to_2005 = projections[["Country","Average"]].rename(columns={"Average":"AVG_1986_to_2005"}).reset_index(drop=True)
PROJECTION_2020_to_2039 = projections[["Country","5th","50th","95th"]].reset_index(drop=True)
PROJECTION_2040_to_2059 = projections[["Country","5th.1","50th.1","95th.1"]].rename(columns={"5th.1":"5th","50th.1":"50th","95th.1":"95th"}).reset_index(drop=True)
PROJECTION_2080_to_2099 = projections[["Country","5th.2","50th.2","95th.2"]].rename(columns={"5th.2":"5th","50th.2":"50th","95th.2":"95th"}).reset_index(drop=True)

# Example of updated dataframe
PROJECTION_2040_to_2059.head(3)

### Explotary Data Analysis: Climate Projections

In [ ]:
# Analyzing the datasets using the describe function to get the statistical summaries

print('\033[1m'+'Actual AVG from 1986 to 2005'+'\033[0m')
print(AVG_1986_to_2005.describe().round(2))
print('')
print('\033[1m'+'2020 to 2039'+'\033[0m')
print(PROJECTION_2020_to_2039.describe().round(2))
print('')
print('\033[1m'+'2040 to 2059'+'\033[0m')
print(PROJECTION_2040_to_2059.describe().round(2))
print('')
print('\033[1m'+'2080 to 2099'+'\033[0m')
print(PROJECTION_2080_to_2099.describe().round(2))

In [ ]:
# Percentage change of projected temperature increase
# Selecting the columns to be used
countries = AVG_1986_to_2005[['Country']]
avg_change1986_2005 = AVG_1986_to_2005[['AVG_1986_to_2005']].rename(columns={'AVG_1986_to_2005':'1986-2005'})
avg_change2020_2039 = PROJECTION_2020_to_2039[['50th']].rename(columns={'50th':'2020-2039'})
avg_change_2040_2059 = PROJECTION_2040_to_2059[['50th']].rename(columns={'50th':'2040-2059'})
avg_change_2080_2099 = PROJECTION_2080_to_2099 [['50th']].rename(columns={'50th':'2080-2099'})

# Using the function "concat" to create dataframe
percentage_change = pd.concat([avg_change1986_2005,avg_change2020_2039,avg_change_2040_2059,avg_change_2080_2099], 
                              axis = 1, sort=False).apply(pd.to_numeric)

# Using the function "pct_change" to calculate percentage change
pc = percentage_change.pct_change(axis='columns')*100
percentage_change = pd.concat([countries,pc],axis =1, sort=False)

# Printing Results
percentage_change.head(3)

In [ ]:
# Calculating cumulative projected temperature increase 
percentage_change['Projected % Cumulative Temperature Increase by 2099'] = percentage_change.sum(axis=1)

# Returning the top 15 countries with the highest projected increase in temperature
top_15 = percentage_change.nlargest(15,'Projected % Cumulative Temperature Increase by 2099')

# Printing Results
top_15.head(15)

In [ ]:
# Ploting: Projected % Comulative Temperature Increase by 2099

plt.figure(figsize=(10,5))
plt.bar(top_15['Country'], top_15['Projected % Cumulative Temperature Increase by 2099'])
plt.xticks(rotation=90)
plt.xlabel('Country Name')
plt.ylabel('Cumulative % Increase')
plt.title('Countries With Highest Projected % Cumulative Temperature Increase by 2099')
plt.show()



### Exploratory Data Analysis: Climate Projections - Results

This dataset contains information about the historical global surface temperature data for 1986-2005 and future temperature projections from 2020-2039 (the next 20 years), 204-2059 (mid-century), and 2080-2099 (late-century). Each country has an average temperature value for the respective ranges of years. The bar plot above shows the top 15 countries with the largest projected percent temperature increase. These countries are the ones that are most likely going to face the severe challenges of the climate crisis. In decreasing order these are the top 15 countries with the most projected temperature increase:


1. Netherlands Antilles - 20.1%
2. Greenland - 20.0%
3. Svalbard and Jan Mayen - 19.7%
4. Liechtenstein - 17.8%
5. Lesotho - 17.5%
6. Macedonia - 17.4%
7. Kosovo - 17.3%
8. Kyrgystan - 17.1%
9. Canada - 17.1%
10. Bouvet Island - 17.0%
11. Bosnia and Herzegovina - 16.7%
12. Armenia - 16.7%
13. Montenegro - 16.6%
14. San Marino - 16.6%
15. Austria - 16.5%

All of these countries have similar projected temperature increases. It is interesting to notice that many of these places are islands and in the northern part of the earth.

### Data Aquisition: Global Glacier Recession


5. GLIMS Glacier Database/ Contributions to Sea Level Rice: Largest Contributors to Global Water Cycle and Sea Level Rise by Reegion. Dataset contains the percentage of total Area and the percentage of contribution to volume change from 1961 to 2003. This analysis is focused on mountain glaciers and smaller ice caps, which have a total area at least 785x103 km2. Although they make up only 4% of the total land ice area, they may have contributed to as much as 30% of sea level change in the 20th century due to rapid ice volume reduction connected with global warming. The data needed was collected from this source via Web Scraping. 

**NSIDC: Glacier Melt** (https://nsidc.org/glims/glaciermelt).

In [ ]:
# Glaciers web scraping 
tables2 = pd.read_html('https://nsidc.org/glims/glaciermelt')

# Retrieving Data
g2 = tables2[0]

# Printing Results
g2.head(3)

### Data Preparation: Climate Projections

In [ ]:
# Looking for null values
print(g2.isnull().sum())

# Shape of the data
print(g2.shape)

### Explotary Data Analysis: Global Glacier Recession

In [ ]:
# Updating column titles for analysis

g2.columns = range(g2.shape[1])

updated_g2 = g2.rename(columns={0:'Location', 1:'percentage_total_area', 2:'volume_change_1961_2003'})

# Printing updated dataframe
updated_g2.head(6)

In [ ]:
# Visualizing percentage of total area and volume change from 1961 to 2003
ax = updated_g2[['Location', 'volume_change_1961_2003']].plot(x='Location', linestyle='-', marker='o', color='firebrick')
updated_g2[['Location', 'percentage_total_area']].plot(x='Location', kind='bar', color='cornflowerblue',ax=ax)

plt.title('Total Area of the Largest Contributors to Global Water Cycle and their Volume Change from 1961 to 2003')
plt.xlabel('Regions')
plt.ylabel('Percentage of Total Area (blue)/''\n''Percentage of Contribution to Volume Change (red)')

plt.show()

### Exploratory Data Analysis: Global Glacier Reduction - Results

Based on the scatter and bar plot above, this dataset contains information about ther perdecntage total area and the percentage of contribution to the volume change of the sea leavel from 1911 to 2003 for 6 regions. The bar plot shows that the percent change of glacier surface area changed the most in the following regions in decreasing order:

1. Arctic - 52.7%
2. High Mountain Asia - 19.4%
3. Alaska and Costal Mountains - 15.0%
4. NW USA and SW Canada - 6.5%
5. Patagonia and Ice Fields - 3.3%
6. Europe - 3.1%

Following this, the scatter plot above shows that the percent of contribution to the volume change decreases in the same descending order as the surface area. This makes sense, the glaciers that lose more surface area will contribute more to the volume change of the sea level. In decreasing order, the percent volume change of the regions are as follows:

1. Arctic - 31.5%
2. High Mountain Asia - 23.9%
3. Alaska and Costal Mountains - 23.0%
4. NW USA and SW Canada - 16.6%
5. Patagonia and Ice Fields - 4.7%
6. Europe - 0.3%

Summarizing the graphs above, they show that there has been a rapid ice volume reduction. This agrees with the EDA section discussing the Glacier Mass Balance above.

### Exploratory Data Analysis: Surface Temperature Anomaly & Glaciers Mean Cumulative Mass Balance


In [ ]:
# Updating Column Titles
read = read.rename(columns={'Surface temperature anomaly':'Surface_temperature_anomaly'})

# Filtering values - years before 1945
read1945 = read[(read.Year >= 1945)&(read.Year <= 2014)]

# Grouping by year to calculate avg temperature anomaly
read_mean = read1945.groupby(['Year']).Surface_temperature_anomaly.agg(['mean']).reset_index()

# Printing Results
read_mean.head(3)

In [ ]:
# Converting decimals in percentages 
glacier_mean = glacier[['Mean cumulative mass balance']]/100

# Plotting relationship between "Surface Temperature Anomaly" and "Mean cumulative mass balance" 

x = read_mean[['Year']]
y1= read_mean[['mean']]
y2= glacier_mean[['Mean cumulative mass balance']]


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(x,y1, color='r')
ax2.plot(x,y2, color='b')
ax1.set_xlabel('Year')
ax1.set_ylabel('Change in Mean Cumulative Mass Blance', color='b')
ax2.set_ylabel('Change in Temperature Anomaly', color='r')

plt.title('Relation Between Temperature Anomalies and Glacier Mass Balance from 1945 to 2014', fontsize=11)
plt.show()


In [ ]:
# Understanding Relationship between temperature anomalies and glaciers mass change 
read_m = read_mean[['mean']]
relationship = pd.merge(read_m, glacier_mean, left_index=True, right_index=True).rename(columns={'mean':'temp_mean','Mean cumulative mass balance':'glacier_mean'})

# Calculating Standard Correlation Coefficient
corr = relationship['temp_mean'].corr(relationship['glacier_mean'])
corr

### Exploratory Data Analysis: Surface Temperature Anomaly vs. Glacier Mass Balance - Results

Based on the line plot above, this compares the information about global surface temperature anomalies (in red) and the glacier mass balance (in blue). This graph shows a steady increase in the global surface temperature over time and a steady decrease in glacier mass balance. Both have a correlation value of **-0.86**. This suggests that there is a strong negative relationship between the surface temperature anomaly and the glacier mass. This kind of relationship was expected. It is interesting to notice that when the temperature anomaly started steeply increasing, around 1990, the glacier mass balance also starts to steeply decrease.

### Exploratory Data Analysis: Surface Temperature Anomaly & Deaths Caused By Natural Disasters

In [ ]:
# Stablishing the lenght of the years for the analysis
read1990 = read[(read.Year >= 1990)&(read.Year <= 2017)]
# Grouping by year and calculating the mean
read_1990 = read1990.groupby(['Year']).Surface_temperature_anomaly.agg(['mean']).reset_index()

# Printing updated dataframe
read_1990.head(3)

In [ ]:
# Plotting relationship between temperature anomalies and deaths caused by natural disasters
x = read_1990[['Year']]
y1= read_1990[['mean']]
y2= total_by_country[['sum']]


fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(x,y1, color='r')
ax2.plot(x,y2, color='g')
ax1.set_xlabel('Year')
ax1.set_ylabel('Change in Temperature Anomaly', color='r')
ax2.set_ylabel('Deaths By Natural Disasters', color='g')

plt.title('Relation Between Temperature Anomalies and Deaths By Natural Disasters from 1990 to 2017', fontsize=10)
plt.show()


In [ ]:
# Selecting columns to perform the analysis
read_1990 = read_1990[['mean']].reset_index(drop=True)
total_1990 = total_by_country[['sum']].reset_index(drop=True)


# Combining both values in the same dataframe
relation = pd.merge(read_1990, total_1990, left_index=True, right_index=True).rename(columns={'mean':'temp_mean','sum':'total_deaths'})

# Calculating Standard Correlation Coefficient
corr1 = relation['temp_mean'].corr(relation['total_deaths'])

# Printing results
corr1

### Exploratory Data Analysis: Surface Temperature Anomaly vs. Deaths by Natrural Disasters - Results

Based on the line plot above, this compares the information about global surface temerpature anomalies (in red) and the deaths by natural disasters (in green). This graph shows a steady increase in the global surface temperature over time and a few strong peaks of deaths amongst a relatively consistent number of deaths. The correlation value between both of these variables is -0.03. This suggests that there is a weak negative relationship between the surface temperature anomaly and the number of death by natural disasters. It is interesting to notice some of the temperature peaks somewhat correlate with the natureal disasters, but not exactly. It is important to keep in mind that the weak negative correlation between the variables does not indication that there is no secondary relationship at all. Correlation does not imply causation after all.

## Phase 2 - Visualization with GeoPandas

To visualize global data serparared by many locations such as country, a map is the most intuative method. Both the Global Surface Temperature Anomaly Data from the GISTEMP analysis by NASA Goddard Institute of Space Studie and the Climate Projections Data form the Climate Impact Lab have temperature data that would be well illustrated in a mpa format. To do this, the GeoPandas package comes in handy.

**GeoPandas:** The GeoPandas package is an open-source project to make working with geospatial data in Python easier. It is an extension of the panda’s package to allow spatial operations on geometric types (define a spatial location and an association geometric type). It allows the user to do GIS. 

### Global Surface Temperature Anomaly Map

The Global Surface Temperature Anomaly data is organized by year and country. To map this data the country information will be joined to a shape file in a GeoPandas dataframe. This contains information about the surface temperature anomaly data and the geospatial data needed to create a map. The process for this is outlined in the code below.

**Shapefile:** A shapefile is a geospatial vector data format for geographic information software (GIS). This shapefile was sources from ___ and contains information about countries, their two and three digit codes, country names in various languages, continent information, square kilometers, latitude and longitude information in a geometry format, among some other information. This type of file can be read by the geopandas package to help make mapping easier.

The first step is to load the shapefile into a geodataframe and confirm that it is a GeoDataFrame.

In [ ]:
#read the shapefile using geopandas
fp = '/Users/Puji/Desktop/countries_shp/countries.shp'

map_df = gpd.read_file(fp)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
type(map_df)

In [ ]:
map_df.head()

Now that we have the shapefile and the surface temperature anomaly data, we can join the dataframes. Here we will use the three digit country code to do this join. For the GISTEMP data that is the `Code` column, for the shapefile that is the `ISO3` column.

In [ ]:
#join the geodataframe with the cleaned up csv dataframe
merged = map_df.set_index('ISO3').join(read.set_index('Code'))
merged.columns

In [ ]:
merged['Surface_temperature_anomaly'].min()

With this combined dataframe, we can create a function to isolate the data by year and graph it with a slider. Additionally, the slider range and step can be adjusted as desired. This can be seen in the map plot below.

In [ ]:
#create dataframe organized by year
mergedworld = pd.DataFrame()
for i in merged['Year'].unique():
    mergetemp=merged[merged['Year']==i]
    mergedworld = mergedworld.append(mergetemp)
    
mergedworld

#create function that brings a geodataframe with a particular year value
def worldplot(date):
    map_year = merged.loc[merged['Year'] == date]
    map_year.plot('Surface_temperature_anomaly', vmin=-5, vmax=4, cmap = 'hot', figsize=(22,11), legend = True)

#draw an interactive plot for a range of years
#the year range is from 1881 to 2017
#here we are looking at 2000 to 2017
interact(worldplot, date=(1978,2017,3))

### Global Surface Temperature Anomaly: Results

We know from our Exploratory Data Analysis that the global surface temperature anomaly shows an increasing trend from the years of 1881 to 2017. The scatter plot illustrating the Average Surface Temperature Anomaly by year tells us that there is a steep increase of this values from 1980 to 2017.

Using the map above to view the temperature anomalies globally over a range of year further illustrates this pattern. For example, setting the slider from 1978 to 2017 with a step of 3 shows how the surface temperature has drastically increased in this time.


### Global Surface Temperature Projection Maps


The Global Surface Temperature Projection data is organized by country name and temperature values. To map this data the country name information will be joined to the country name in the shapefile GeoPandas dataframe. This contains information about the historical and projected surface temperature and the geospatial data from the shapefile is needed to plot the map. There are four separate dataframes that sum up the projections for this century; the first one is historical from 1986 to 2005, the other three dataframe are calculated projections spanning the following year ranges: 2020-2039 2040-2059, and 2080-2099. The process for this is outlined in the code below.

The first step is to join the dataframes to the historical temperature dataframe. The will be joined on the `NAME` column on the shapefile and the `Average` column for the historical data. After joining, then we have to check if there are any null values.

In [ ]:
AVG_1986_to_2005 = AVG_1986_to_2005.rename(columns={'AVG_1986_to_2005':'Average'})
AVG_1986_to_2005.head()

In [ ]:
geo8605 = map_df.set_index('NAME').join(AVG_1986_to_2005.set_index('Country'))
geo8605_null= geo8605[geo8605['Average'].isna()]
geo8605_null

We can see that there are null values. This is because the historical and projections temperature data only have country names and no identifying code. Some country names are written differently and thus they are not joined together. This can be fixed manually by replacing these country names and rejoining the dataframes.

In [ ]:
AVG_1986_to_2005['Country'] = AVG_1986_to_2005['Country'].replace(
    ['Antarctica','Democratic Republic of the Congo','Cura�ao','Falkland Islands','Libyan Arab Jamahiriya',
     'Democratic People`s Republic of Korea (North Korea)','Syrian Arab Republic','United Republic of Tanzania',
     'Vatican','Virgin Islands (US)', 'Republic of Korea (South Korea)', 'United States of America', 
     'Russian Federation','Viet Nam', 'United Republic of Tanzania', 'R�union', 'Pitcairn', 'Myanmar'],
    ['Antarctica','Congo (Democratic Republic of the)','Curaçao','Falkland Islands (Islas Malvinas)','Libya',
     'North Korea','Syria','Tanzania','Vatican City','Virgin Islands', 'South Korea', 'United States', 
     'Russia', 'Vietnam', 'Tanzania', 'Reunion', 'Pitcairn Islands', 'Burma'])

After this, we can change the data type of the temperature column to float. This makes the plot legend in the form of a color bar. Which is what we want.

The rest of the projected temperature dataframes follow the same process of joining with the shapefile and adjusting the country names.

In [ ]:
geo8605 = map_df.set_index('NAME').join(AVG_1986_to_2005.set_index('Country'))
geo8605['Average'] = geo8605['Average'].astype('float')


In [ ]:
PROJECTION_2020_to_2039.head()

In [ ]:
geo2039 = map_df.set_index('NAME').join(PROJECTION_2020_to_2039.set_index('Country'))
geo2039_null= geo2039[geo2039['50th'].isna()]
geo2039_null

In [ ]:
PROJECTION_2020_to_2039['Country'] = PROJECTION_2020_to_2039['Country'].replace(
    ['Antarctica','Democratic Republic of the Congo','Cura�ao','Falkland Islands','Libyan Arab Jamahiriya',
     'Democratic People`s Republic of Korea (North Korea)','Syrian Arab Republic','United Republic of Tanzania',
     'Vatican','Virgin Islands (US)', 'Republic of Korea (South Korea)', 'United States of America', 
     'Russian Federation','Viet Nam', 'United Republic of Tanzania', 'R�union', 'Pitcairn', 'Myanmar'],
    ['Antarctica','Congo (Democratic Republic of the)','Curaçao','Falkland Islands (Islas Malvinas)','Libya',
     'North Korea','Syria','Tanzania','Vatican City','Virgin Islands', 'South Korea', 'United States', 
     'Russia', 'Vietnam', 'Tanzania', 'Reunion', 'Pitcairn Islands', 'Burma'])

In [ ]:
geo2039 = map_df.set_index('NAME').join(PROJECTION_2020_to_2039.set_index('Country'))
geo2039['50th'] = geo2039['50th'].astype('float')


In [ ]:
PROJECTION_2040_to_2059.head()

In [ ]:
geo4059 = map_df.set_index('NAME').join(PROJECTION_2040_to_2059.set_index('Country'))
geo4059_null= geo4059[geo4059['50th'].isna()]
geo4059_null

In [ ]:
PROJECTION_2040_to_2059['Country'] = PROJECTION_2040_to_2059['Country'].replace(
    ['Antarctica','Democratic Republic of the Congo','Cura�ao','Falkland Islands','Libyan Arab Jamahiriya',
     'Democratic People`s Republic of Korea (North Korea)','Syrian Arab Republic','United Republic of Tanzania',
     'Vatican','Virgin Islands (US)', 'Republic of Korea (South Korea)', 'United States of America', 
     'Russian Federation','Viet Nam', 'United Republic of Tanzania', 'R�union', 'Pitcairn', 'Myanmar'],
    ['Antarctica','Congo (Democratic Republic of the)','Curaçao','Falkland Islands (Islas Malvinas)','Libya',
     'North Korea','Syria','Tanzania','Vatican City','Virgin Islands', 'South Korea', 'United States', 
     'Russia', 'Vietnam', 'Tanzania', 'Reunion', 'Pitcairn Islands', 'Burma'])

In [ ]:
geo4059 = map_df.set_index('NAME').join(PROJECTION_2040_to_2059.set_index('Country'))
geo4059['50th'] = geo4059['50th'].astype('float')


In [ ]:
PROJECTION_2080_to_2099.head()

In [ ]:
geo8099 = map_df.set_index('NAME').join(PROJECTION_2080_to_2099.set_index('Country'))
geo8099_null= geo8099[geo8099['50th'].isna()]
geo8099_null

In [ ]:
PROJECTION_2080_to_2099['Country'] = PROJECTION_2080_to_2099['Country'].replace(
    ['Antarctica','Democratic Republic of the Congo','Cura�ao','Falkland Islands','Libyan Arab Jamahiriya',
     'Democratic People`s Republic of Korea (North Korea)','Syrian Arab Republic','United Republic of Tanzania',
     'Vatican','Virgin Islands (US)', 'Republic of Korea (South Korea)', 'United States of America', 
     'Russian Federation','Viet Nam', 'United Republic of Tanzania', 'R�union', 'Pitcairn', 'Myanmar'],
    ['Antarctica','Congo (Democratic Republic of the)','Curaçao','Falkland Islands (Islas Malvinas)','Libya',
     'North Korea','Syria','Tanzania','Vatican City','Virgin Islands', 'South Korea', 'United States', 
     'Russia', 'Vietnam', 'Tanzania', 'Reunion', 'Pitcairn Islands', 'Burma'])

In [ ]:
geo8099 = map_df.set_index('NAME').join(PROJECTION_2080_to_2099.set_index('Country'))
geo8099['50th'] = geo8099['50th'].astype('float')


After the appropriate dataframes are created for historical, next 20 year, mid-century, and late-century temperature values, a function can be written to plot these graphs. This function takes an input from the slider bar and plots the appopriate map. Both the map and the function are in the cell below.

In [ ]:
project_mergedworld = pd.DataFrame()

projections_plots = (geo8605, geo2039, geo4059, geo8099)

def projections_worldplot(date):
    if date == 1:
        geo8605.plot('Average', vmin=0, vmax=110, cmap='hot', figsize=(22,11), legend = True)
    if date == 2:
        geo2039.plot('50th', vmin=0, vmax=110, cmap='hot', figsize=(22,11), legend = True)
    if date == 3:
        geo4059.plot('50th', vmin=0, vmax=110, cmap='hot', figsize=(22,11), legend = True)
    if date == 4:
        geo8099.plot('50th', vmin=0, vmax=110, cmap='hot', figsize=(22,11), legend = True)
        
#interact(worldplot, date=range(1880,2018))
interact(projections_worldplot, date=(1,4,1))

### Global Surface Temperature Projection Map: Results

We know from our Exploratory Data Analysis that the global surface temerpature anomaly shows an increasing trend from the for the ranges of years provided. Using the map above to view the global temperature projections over a range of years for the century further illustrates this pattern. The slider takes us through the following timeline:

1. Historical: 1986 - 2005
2. The Next 20 Years: 2020 - 2039
3. Mid-Century: 2040 - 2059
4. Late-Century: 2089 - 2099.

The increase in global temperature can be seen by the increase in yellow in the map above.

## Results and Conclusions

From the EDA and the Geoplots in the previous sections we can answer our research questions.

**What are the trends in global surface temperature?**
The trends in global surface temperature have shown a steady increase from 1986 to 2017 with a particularly steep rate of increase from 1980 onwards. Based on future climate predictions, there will continue to be an increase in global surface temperature, continuing this trend until at least the end of the century.

**How do global temperature trends affect glaciers?**
The increase in global temperature decreases the mean glacier mass. These two variable have a large negative correlation. This is expected and due to the continuing increase in surface temperature, the glacier mass will continue to lessen. The glaciers in the arctic region have decrease in surface area the most and contribute the most to the percent volume increase in the sea level.

**How do global temperature trends affect the number of deaths by natural disasters?**
The global surface temperature anomaly has a weak negative correlation with the deaths by natural disasters. Although these are not strongly correlated, this does not mean they are unconnected. There are some significant natural disasters that occur during temperature spikes. This would be an interested area to continue research in.

This type of work and visualization would be greatly beneficial for climate scientists, environmental agencies, those in natural disaster prevention industries, and anyone interested in learning about climate data, amongst others. This project can be further expanded by looking at more climate variable and finding more relationships between global temperature trends and metrics that impact human life.
